In [1]:
import tensorflow as tf
from tensorflow.keras import layers, losses, models

import numpy as np
import matplotlib.pyplot as plt


# Get MNIST data

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train / 255., axis=-1)
x_test = np.expand_dims(x_test / 255., axis=-1)

print("Number of training data: {}".format(len(x_train)))
print("Number of test data: {}".format(len(x_test)))
print()
print("x shape: {}".format(x_train.shape))
print("y shape: {}".format(y_train.shape))
print()
print("x range: {}".format([x_train.min(), x_train.max()]))
print("x type: {}".format(x_train.dtype))
print("Unique labels: {}".format(np.unique(y_train)))

Number of training data: 60000
Number of test data: 10000

x shape: (60000, 28, 28, 1)
y shape: (60000,)

x range: [0.0, 1.0]
x type: float64
Unique labels: [0 1 2 3 4 5 6 7 8 9]


# Build a convolutional autoencoder

In [3]:
class CAE(models.Model):
    
    def __init__(self, input_shape, num_filters_list, latent_dim):
        super(CAE, self).__init__()
        self.latent_dim = latent_dim
        
        self.encode_net = tf.keras.Sequential(
            [layers.InputLayer(input_shape=input_shape)] + \
            [self.encoder(n) for n in num_filters_list] + \
            [layers.Flatten(),
             layers.Dense(latent_dim)]
        )
        
        r = tf.cast(input_shape[0] / 2**len(num_filters_list), tf.int32)
        c = tf.cast(input_shape[1] / 2**len(num_filters_list), tf.int32)
        decode_num_filters_list = [n for n in reversed(num_filters_list[:-1])] + [input_shape[-1]]
        
        self.decode_net = tf.keras.Sequential(
            [layers.Dense(r * c * num_filters_list[-1]),
             layers.Reshape(target_shape=(r, c, num_filters_list[-1]))] + \
            [self.decoder(n) for n in decode_num_filters_list]
        )
        
    def encoder(self, num_filters):
        return tf.keras.Sequential([
            layers.Conv2D(num_filters, (3, 3), padding='same', activation='relu'),
            layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2))
        ])
    
    def decoder(self, num_filters):
        return layers.Conv2DTranspose(num_filters, kernel_size=(2, 2), strides=(2, 2), activation='relu')
    
    
    def call(self, inputs):
        z = self.encode_net(inputs)
        return self.decode_net(z)
    
    def encode(self, x):
        return self.encode_net(x)
    
    def decode(self, z):
        return self.decode_net(z) 

# Train the model

In [4]:
model = CAE(input_shape=(28, 28, 1), num_filters_list=[32, 64], latent_dim=30)
model.compile(optimizer='adam', loss=losses.mean_squared_error)

In [5]:
cp = tf.keras.callbacks.ModelCheckpoint('tmp/weights-{epoch:04d}.ckpt', verbose=1,
                                        save_best_only=True, save_weights_only=True)
history = model.fit(x_train, x_train, batch_size=32, epochs=20, validation_split=0.2, callbacks=[cp])

Train on 48000 samples, validate on 12000 samples
Epoch 1/20
48000/48000 [==============================] - 16s 326us/sample - loss: 0.0157 - val_loss: 0.0084
Epoch 2/20
48000/48000 [==============================] - 14s 294us/sample - loss: 0.0073 - val_loss: 0.0069
Epoch 3/20
48000/48000 [==============================] - 14s 297us/sample - loss: 0.0065 - val_loss: 0.0063
Epoch 4/20
48000/48000 [==============================] - 14s 297us/sample - loss: 0.0061 - val_loss: 0.0061
Epoch 5/20
48000/48000 [==============================] - 14s 295us/sample - loss: 0.0058 - val_loss: 0.0058
Epoch 6/20
48000/48000 [==============================] - 14s 295us/sample - loss: 0.0056 - val_loss: 0.0057
Epoch 7/20
48000/48000 [==============================] - 14s 300us/sample - loss: 0.0055 - val_loss: 0.0057
Epoch 8/20
48000/48000 [==============================] - 14s 301us/sample - loss: 0.0054 - val_loss: 0.0055
Epoch 9/20
48000/48000 [==============================] - 14s 299us/sample - l

In [ ]:
%matplotlib notebook
x_pred = model(X2[:5])
for i in range(len(x_pred)):   
    
    plt.subplot(5,2,2*i+1)
    plt.imshow(X2[i,...,0], cmap='gray')
    
    plt.subplot(5,2,2*i+2)
    plt.imshow(x_pred[i,...,0], cmap='gray')